In [ ]:
# 1. Tải và cài đặt Google Chrome (Phiên bản Stable)
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get -y update
!apt-get install -y ./google-chrome-stable_current_amd64.deb

# 2. Cài đặt Selenium và Webdriver Manager (Tự động tìm driver phù hợp)
!pip install selenium webdriver-manager

--2025-12-17 15:44:20--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.10.136, 142.251.10.91, 142.251.10.93, ...
Connecting to dl.google.com (dl.google.com)|142.251.10.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117816476 (112M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 112.36M   158MB/s    in 0.7s    

2025-12-17 15:44:21 (158 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [117816476/117816476]

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes

In [ ]:

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# --- 1. CẤU HÌNH TRÌNH DUYỆT ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

print("🚀 Đang khởi động crawler cho dự án cuối kỳ...")
try:
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    print("✅ Trình duyệt sẵn sàng!")
except Exception as e:
    print(f"❌ Lỗi driver: {e}")

# --- 2. THIẾT LẬP THAM SỐ (PHẦN QUAN TRỌNG) ---
# Link chuyên mục: CNTT - Phần mềm (c1)
# Cấu trúc link: https://careerviet.vn/viec-lam/cntt-phan-mem-c1-trang-{}-vi.html
base_url = "https://careerviet.vn/viec-lam/cntt-phan-mem-c1-trang-{}-vi.html"

# SỐ TRANG MUỐN LẤY: Để làm đồ án, bạn nên lấy khoảng 30-50 trang
so_trang_muon_lay = 50

data_list = []
print(f"🎯 Mục tiêu: Crawl {so_trang_muon_lay} trang thuộc nhóm ngành CNTT...")

# --- 3. BẮT ĐẦU CRAWL ---
for page in range(1, so_trang_muon_lay + 1):
    # Tạo link cho từng trang
    if page == 1:
        url = "https://careerviet.vn/viec-lam/cntt-phan-mem-c1-vi.html"
    else:
        url = base_url.format(page)

    print(f"   ⏳ Đang xử lý trang {page}/{so_trang_muon_lay}...", end="\r") # In đè dòng để gọn

    try:
        driver.get(url)
        time.sleep(2) # Nghỉ 2s mỗi trang để tránh bị chặn

        jobs = driver.find_elements(By.CSS_SELECTOR, ".job-item")

        for job in jobs:
            try:
                # Lấy Title
                try: title = job.find_element(By.CSS_SELECTOR, ".title a").text
                except: title = ""

                # Lấy Company
                try: company = job.find_element(By.CSS_SELECTOR, ".company-name").text
                except: company = ""

                # Lấy Salary
                try: salary = job.find_element(By.CSS_SELECTOR, ".salary p").text
                except: salary = "Thỏa thuận"

                # Lấy Location
                try: location = job.find_element(By.CSS_SELECTOR, ".location").text
                except: location = "Vietnam"

                if title:
                    data_list.append({
                        "Job Title": title,
                        "Company": company,
                        "Salary": salary,
                        "Location": location
                    })
            except:
                continue

    except Exception as e:
        print(f"\n❌ Lỗi tại trang {page}: {e}")

driver.quit()

# --- 4. LƯU KẾT QUẢ ---
print(f"\n\n✅ HOÀN THÀNH! Tổng số job thu thập được: {len(data_list)}")

if len(data_list) > 0:
    df = pd.DataFrame(data_list)

    # Loại bỏ các dòng trùng lặp (nếu có)
    df.drop_duplicates(inplace=True)
    print(f"📉 Sau khi loại bỏ trùng lặp còn: {len(df)} jobs")

    # Lưu file
    file_name = "jobs_it_final.csv"
    df.to_csv(file_name, index=False, encoding="utf-8-sig")
    print(f"📁 Đã lưu file: {file_name}")

    # Hiển thị thống kê nhanh
    print("\n--- Thống kê sơ bộ ---")
    print(df['Location'].value_counts().head())
else:
    print("❌ Không lấy được dữ liệu.")

🚀 Đang khởi động crawler cho dự án cuối kỳ...
✅ Trình duyệt sẵn sàng!
🎯 Mục tiêu: Crawl 50 trang thuộc nhóm ngành CNTT...


✅ HOÀN THÀNH! Tổng số job thu thập được: 1314
📉 Sau khi loại bỏ trùng lặp còn: 994 jobs
📁 Đã lưu file: jobs_it_final.csv

--- Thống kê sơ bộ ---
Location
Hà Nội                 522
Hồ Chí Minh            359
Bình Dương              27
Hà Nội\nHồ Chí Minh     12
Đồng Nai                10
Name: count, dtype: int64


In [ ]:

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# --- 1. CẤU HÌNH TRÌNH DUYỆT ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

print("🚀 Đang khởi động crawler cho dự án cuối kỳ...")
try:
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    print("✅ Trình duyệt sẵn sàng!")
except Exception as e:
    print(f"❌ Lỗi driver: {e}")

# --- 2. THIẾT LẬP THAM SỐ (PHẦN QUAN TRỌNG) ---
# Link chuyên mục: CNTT - Phần mềm (c1)
# Cấu trúc link: https://careerviet.vn/viec-lam/cntt-phan-mem-c1-trang-{}-vi.html
base_url = "https://careerviet.vn/viec-lam/cntt-phan-mem-c1-trang-{}-vi.html"

# SỐ TRANG MUỐN LẤY: Để làm đồ án, bạn nên lấy khoảng 30-50 trang
so_trang_muon_lay = 200

data_list = []
print(f"🎯 Mục tiêu: Crawl {so_trang_muon_lay} trang thuộc nhóm ngành CNTT...")

# --- 3. BẮT ĐẦU CRAWL ---
for page in range(1, so_trang_muon_lay + 1):
    # Tạo link cho từng trang
    if page == 1:
        url = "https://careerviet.vn/viec-lam/cntt-phan-mem-c1-vi.html"
    else:
        url = base_url.format(page)

    print(f"   ⏳ Đang xử lý trang {page}/{so_trang_muon_lay}...", end="\r") # In đè dòng để gọn

    try:
        driver.get(url)
        time.sleep(2) # Nghỉ 2s mỗi trang để tránh bị chặn

        jobs = driver.find_elements(By.CSS_SELECTOR, ".job-item")

        for job in jobs:
            try:
                # Lấy Title
                try: title = job.find_element(By.CSS_SELECTOR, ".title a").text
                except: title = ""

                # Lấy Company
                try: company = job.find_element(By.CSS_SELECTOR, ".company-name").text
                except: company = ""

                # Lấy Salary
                try: salary = job.find_element(By.CSS_SELECTOR, ".salary p").text
                except: salary = "Thỏa thuận"

                # Lấy Location
                try: location = job.find_element(By.CSS_SELECTOR, ".location").text
                except: location = "Vietnam"

                if title:
                    data_list.append({
                        "Job Title": title,
                        "Company": company,
                        "Salary": salary,
                        "Location": location
                    })
            except:
                continue

    except Exception as e:
        print(f"\n❌ Lỗi tại trang {page}: {e}")

driver.quit()

# --- 4. LƯU KẾT QUẢ ---
print(f"\n\n✅ HOÀN THÀNH! Tổng số job thu thập được: {len(data_list)}")

if len(data_list) > 0:
    df = pd.DataFrame(data_list)

    # Loại bỏ các dòng trùng lặp (nếu có)
    df.drop_duplicates(inplace=True)
    print(f"📉 Sau khi loại bỏ trùng lặp còn: {len(df)} jobs")

    # Lưu file
    file_name = "jobs_it_final.csv"
    df.to_csv(file_name, index=False, encoding="utf-8-sig")
    print(f"📁 Đã lưu file: {file_name}")

    # Hiển thị thống kê nhanh
    print("\n--- Thống kê sơ bộ ---")
    print(df['Location'].value_counts().head())
else:
    print("❌ Không lấy được dữ liệu.")

🚀 Đang khởi động crawler cho dự án cuối kỳ...
✅ Trình duyệt sẵn sàng!
🎯 Mục tiêu: Crawl 200 trang thuộc nhóm ngành CNTT...


✅ HOÀN THÀNH! Tổng số job thu thập được: 1730
📉 Sau khi loại bỏ trùng lặp còn: 10 jobs
📁 Đã lưu file: jobs_it_final.csv

--- Thống kê sơ bộ ---
Location
Hà Nội                 6
Hà Tĩnh                2
Quảng Nam | Đà Nẵng    1
Long An                1
Name: count, dtype: int64


##crawl lại

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# --- 1. CẤU HÌNH TRÌNH DUYỆT ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

print("🚀 Đang khởi động crawler cho danh mục CNTT (Phần cứng & Phần mềm)...")
try:
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    print("✅ Trình duyệt sẵn sàng!")
except Exception as e:
    print(f"❌ Lỗi driver: {e}")
    exit()

# --- 2. THIẾT LẬP THAM SỐ ---
base_url_page_1 = "https://careerviet.vn/viec-lam/cntt-phan-cung-mang-cntt-phan-mem-c63,1-vi.html"
base_url_other = "https://careerviet.vn/viec-lam/cntt-phan-cung-mang-cntt-phan-mem-c63,1-trang-{}-vi.html"

so_trang_muon_lay = 23
data_list = []
last_successful_page = 0

print(f"🎯 Mục tiêu: Crawl tối đa {so_trang_muon_lay} trang...")

# --- 3. BẮT ĐẦU CRAWL ---
for page in range(1, so_trang_muon_lay + 1):
    if page == 1:
        url = base_url_page_1
    else:
        url = base_url_other.format(page)

    print(f"📄 Đang crawl trang {page}...")

    try:
        driver.get(url)
        time.sleep(2)

        jobs = driver.find_elements(By.CSS_SELECTOR, ".job-item")

        if not jobs:
            print(f"   ⚠️ Trang {page}: Không tìm thấy việc làm. DỪNG CRAWL tại đây.")
            break

        job_count = 0
        for job in jobs:
            try:
                title_elem = job.find_elements(By.CSS_SELECTOR, ".title a")
                title = title_elem[0].text if title_elem else ""
                if not title.strip():
                    continue

                company_elem = job.find_elements(By.CSS_SELECTOR, ".company-name")
                company = company_elem[0].text if company_elem else ""

                salary_elem = job.find_elements(By.CSS_SELECTOR, ".salary p")
                salary = salary_elem[0].text if salary_elem else "Thỏa thuận"

                location_elem = job.find_elements(By.CSS_SELECTOR, ".location")
                location = location_elem[0].text if location_elem else "Vietnam"

                data_list.append({
                    "Job Title": title,
                    "Company": company,
                    "Salary": salary,
                    "Location": location
                })
                job_count += 1

            except Exception:
                continue

        print(f"   ✅ Trang {page}: Lấy được {job_count} job(s).")
        last_successful_page = page  # Ghi lại trang cuối thành công

    except Exception as e:
        print(f"   ❌ Lỗi khi crawl trang {page}: {e}")
        break

# --- 4. KẾT THÚC ---
driver.quit()

print(f"\n✅ HOÀN THÀNH!")
print(f"🔗 Đã crawl thành công đến trang: {last_successful_page} / {so_trang_muon_lay}")
print(f"📥 Tổng số job thu thập được: {len(data_list)}")

if data_list:
    df = pd.DataFrame(data_list)
    df.drop_duplicates(subset=["Job Title", "Company", "Location"], inplace=True)
    print(f"🧹 Sau khi loại bỏ trùng lặp: {len(df)} job(s)")

    file_name = "jobs_it_hardware_software.csv"
    df.to_csv(file_name, index=False, encoding="utf-8-sig")
    print(f"📁 Đã lưu file: {file_name}")

    print("\n--- Top 5 địa điểm ---")
    print(df['Location'].value_counts().head())
else:
    print("❌ Không có dữ liệu.")

🚀 Đang khởi động crawler cho danh mục CNTT (Phần cứng & Phần mềm)...
✅ Trình duyệt sẵn sàng!
🎯 Mục tiêu: Crawl tối đa 23 trang...
📄 Đang crawl trang 1...
   ✅ Trang 1: Lấy được 50 job(s).
📄 Đang crawl trang 2...
   ✅ Trang 2: Lấy được 50 job(s).
📄 Đang crawl trang 3...
   ✅ Trang 3: Lấy được 50 job(s).
📄 Đang crawl trang 4...
   ✅ Trang 4: Lấy được 50 job(s).
📄 Đang crawl trang 5...
   ✅ Trang 5: Lấy được 50 job(s).
📄 Đang crawl trang 6...
   ✅ Trang 6: Lấy được 50 job(s).
📄 Đang crawl trang 7...
   ✅ Trang 7: Lấy được 50 job(s).
📄 Đang crawl trang 8...
   ✅ Trang 8: Lấy được 50 job(s).
📄 Đang crawl trang 9...
   ✅ Trang 9: Lấy được 50 job(s).
📄 Đang crawl trang 10...
   ✅ Trang 10: Lấy được 50 job(s).
📄 Đang crawl trang 11...
   ✅ Trang 11: Lấy được 50 job(s).
📄 Đang crawl trang 12...
   ✅ Trang 12: Lấy được 50 job(s).
📄 Đang crawl trang 13...
   ✅ Trang 13: Lấy được 50 job(s).
📄 Đang crawl trang 14...
   ✅ Trang 14: Lấy được 50 job(s).
📄 Đang crawl trang 15...
   ✅ Trang 15: Lấy được